<a href="https://colab.research.google.com/github/ahmadsameh8/Score_prediction/blob/main/video_subtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# def translate(word):
#   for i in range(len(word)):

#     article = word[i]["text"]

#     inputs = tokenizer_nllp(article, return_tensors="pt")
#     translated_tokens = model_nllp.generate(**inputs, forced_bos_token_id=tokenizer_nllp.lang_code_to_id["arz_Arab"])
#     word[i]["text"] = tokenizer_nllp.batch_decode(translated_tokens, skip_special_tokens=True)[0]
#   return word

In [ ]:
# !pip install subtitle-editor
# !pip install PyAudio
# !sudo apt-get install portaudio19-dev
# !pip install pyaudio
# !pip3 uninstall ffmpeg
# !sudo apt install libmagick++-dev
# !pip3 install ffmpeg-python
# !pip3 install pysrt==1.1.2



In [ ]:
!pip3 install whisper-timestamped
!pip install pydub==0.25.1
!pip install moviepy==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.0/745.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 36.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=baa560c18daa19bb92763ce25a57efcc6f4068880a4289dacaacba79170fcc14
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [ ]:
# import torch
import whisper_timestamped as whisper
import moviepy.editor as mp
# import time
# import math
# import ffmpeg
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import json
from pydub import AudioSegment


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [ ]:
model_nllp_path = "facebook/nllb-200-distilled-600M"
model_whisper = whisper.load_model("base")
tokenizer_nllp = AutoTokenizer.from_pretrained(model_nllp_path)
model_nllp = AutoModelForSeq2SeqLM.from_pretrained(model_nllp_path)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 156MiB/s]
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

  return self.fget.__get__(instance, owner)()



generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def extract_audio(video_file_path):
    """Extracts the audio from a video file and saves it with the same name but with a .wav extension.
    Args:
        video_file_path (str): The path to the video file.

    Returns:
        None
    """
    video = mp.VideoFileClip(video_file_path)


    audio = video.audio

    video_name = video.filename.rsplit('.', 1)[0]

    audio_file_path = f"{video_name}.wav"

    audio.write_audiofile(audio_file_path)

    return audio_file_path

In [ ]:
def split_audio_to_chunks(audio_file):
  """
  Splits an audio file into chunks and saves them as WAV files with numbered filenames.

  Args:
      audio_file (str): Path to the audio file.
      chunk_length (int): Duration of each chunk in milliseconds.

  Returns:
      list: List of paths to the exported WAV files.
  """

  # Open the audio file
  sound = AudioSegment.from_wav(audio_file)

  # Loop through the audio, splitting into chunks and saving
  chunk_paths = []
  chunk_number = 1
  start = 0
  while start < len(sound):
    end = min(start + 10000, len(sound))
    chunk = sound[start:end]

    # Save the chunk as a WAV file with numbered filename
    chunk_filename = f"chunk_{chunk_number}.wav"
    chunk.export(chunk_filename, format="wav")
    chunk_paths.append(chunk_filename)

    chunk_number += 1
    start += 10000

  return chunk_paths

In [ ]:
def transcript(audio_paths):
  """
  This function transcribes audio files specified by audio_paths and returns a list of transcriptions.

  Args:
      audio_paths: A list containing paths to the audio files.

  Returns:
      A list of lists. Each inner list contains dictionaries with 'text', 'start', and 'end' keys for each word in the corresponding audio file.
  """
  transcriptions = []
  for audio_path in audio_paths:
    audio = whisper.load_audio(audio_path)
    result = whisper.transcribe(model_whisper, audio)
    words = []
    for segment in result["segments"]:
      text = segment["text"]
      start = segment["start"]
      end = segment["end"]
      formatted_segment = {'text': text, 'start': start, 'end': end}
      words.append(formatted_segment)
    transcriptions.append(words)
  return transcriptions


In [ ]:
def translate(word):
  time = 0
  translated_transcriptions = []
  for audio_words in word:
    for aw in audio_words:
      inputs = tokenizer_nllp(aw["text"], return_tensors="pt")
      translated_tokens = model_nllp.generate(**inputs, forced_bos_token_id=tokenizer_nllp.lang_code_to_id["arz_Arab"])
      aw["text"] = tokenizer_nllp.batch_decode(translated_tokens, skip_special_tokens=True)[0]
      aw["start"]+=time
      aw["end"]+=time
      translated_transcriptions.append(aw)

    time +=10


In [ ]:
def seconds_to_srt_time(seconds):
  """Converts seconds to SRT time format with milliseconds.

  Args:
      seconds: Time in seconds

  Returns:
      A string in SRT time format (HH:MM:SS,MMM)
  """


  hours = int(seconds // 3600)
  minutes = int((seconds % 3600) // 60)
  seconds = int(seconds % 60)
  milliseconds = int((seconds - int(seconds)) * 1000)
  return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"


def create_srt_file(word):
  """Creates an SRT file from a list of words and their timings.

  Args:
      word: A list of dictionaries with 'start' (seconds), 'end' (seconds), and 'text' keys.

  Raises:
      ValueError: If the 'start' or 'end' key is missing from a dictionary in the list.
  """
  srt_format = []
  for i, entry in enumerate(word, start=1):
    if 'start' not in entry or 'end' not in entry:
      raise ValueError(f"Missing 'start' or 'end' key in entry {i}")
    start_time = seconds_to_srt_time(entry['start'])
    end_time = seconds_to_srt_time(entry['end'])
    srt_format.append(f"{i}\n{start_time} --> {end_time}\n{entry['text'].strip()}\n\n")

  with open("subtitles.srt", "w") as f:
    f.write("".join(srt_format))

  print("Subtitles saved to subtitles.srt")



In [ ]:

input_video ='/content/مصر اليوم - توفيق عكاشة_ أنا مش عارف إمتى ربنا هينزل حل للقضية الفلسطينية (1).mp4'
audio_path = extract_audio(input_video)

chunk_paths = split_audio_to_chunks(audio_path)



MoviePy - Writing audio in /content/مصر اليوم - توفيق عكاشة_ أنا مش عارف إمتى ربنا هينزل حل للقضية الفلسطينية (1).wav


MoviePy - Done.


In [ ]:
word = transcript(chunk_paths)



Detected language: Arabic


100%|██████████| 1000/1000 [00:00<00:00, 1315.65frames/s]


Detected language: Arabic


100%|██████████| 1000/1000 [00:00<00:00, 2519.85frames/s]


Detected language: Arabic


...

100%|██████████| 1000/1000 [00:00<00:00, 2405.88frames/s]


Detected language: Arabic


100%|██████████| 1000/1000 [00:00<00:00, 2614.48frames/s]


Detected language: Arabic


100%|██████████| 920/920 [00:00<00:00, 2967.06frames/s]


In [ ]:
word

[[{'text': 'مصر كان موقف في عنان السما في التصويت',
   'start': 1.02,
   'end': 9.6}],
 [{'text': 'مع روسيا التحدية في مجلس الأم لصالح القرار الروسي',
   'start': 11.56,
   'end': 17.58}],
 [{'text': 'طبعاً أنا أضع الفلس دني نعني أصبحت هي أن',
   'start': 20.0,
   'end': 30.02}],
 [{'text': 'نقطة أساسية فى كل الاجتماعات الرئيسية فى الفتاع السيسي',
   'start': 30.0,
   'end': 37.14},
  {'text': 'وجميع زياراته', 'start': 38.08, 'end': 39.76}],
 [{'text': 'العالمية والأقلمية حلد والتين هي من السواب المسرية القتس الشرقية',
   'start': 40.58,
   'end': 49.32}],
 [{'text': 'عصمة فلصطين', 'start': 50.04, 'end': 53.16},
  {'text': 'هي كل الحجد كلام', 'start': 54.04, 'end': 55.56},
  {'text': 'وردت عن دينة', 'start': 56.56, 'end': 57.68},
  {'text': 'لكن', 'start': 58.88, 'end': 59.34}],
 [{'text': 'يعني أنا مش عرف', 'start': 63.36, 'end': 64.34},
  {'text': 'ربنا إيمته', 'start': 65.32, 'end': 66.32},
  {'text': 'فهي نزلت من حالتي', 'start': 67.88, 'end': 69.38}],
 [{'text': 'لأضافة الفلسطينيي

In [ ]:
srt = translate(word)



In [ ]:
srt

In [ ]:
subtitles = create_srt_file(word)

ValueError: Missing 'start' or 'end' key in entry 1

In [ ]:
pip install

In [ ]:


import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QVBoxLayout, QWidget, QFileDialog

class VideoProcessorApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Video to SRT Converter")
        self.setGeometry(100, 100, 300, 150)

        self.initUI()

    def initUI(self):
        self.centralWidget = QWidget()
        self.setCentralWidget(self.centralWidget)

        self.layout = QVBoxLayout()

        self.btnSelectVideo = QPushButton("Select Video")
        self.btnSelectVideo.clicked.connect(self.selectVideo)
        self.layout.addWidget(self.btnSelectVideo)

        self.btnProcessVideo = QPushButton("Process Video")
        self.btnProcessVideo.clicked.connect(self.processVideo)
        self.layout.addWidget(self.btnProcessVideo)

        self.centralWidget.setLayout(self.layout)

    def selectVideo(self):
        self.videoFilePath, _ = QFileDialog.getOpenFileName(self, "Select Video File")
        print(f"Selected video: {self.videoFilePath}")

    def processVideo(self):
        if hasattr(self, 'videoFilePath'):
            # Here, integrate your functions for processing the video.
            # This is where you would call extract_audio, split_audio_to_chunks,
            # transcribe, translate, and create_srt_file in sequence.
            print(f"Processing video: {self.videoFilePath}")
            # For demonstration purposes, these steps are omitted.
            # You would implement the actual processing here.
            # Example:
            # audio_path = extract_audio(self.videoFilePath)
            # chunk_paths = split_audio_to_chunks(audio_path)
            # for chunk_path in chunk_paths:
            #     words = transcript(chunk_path)
            #     translated_words = translate(words)
            #     create_srt_file(translated_words)
        else:
            print("No video selected")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = VideoProcessorApp()
    window.show()
    sys.exit(app.exec_())
